In [123]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [124]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/caseymarescot/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [125]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [126]:
# Set up HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [127]:
slide_elem.find('div', class_='content_title')

<div class="content_title">Q&amp;A with the Student Who Named Ingenuity, NASA's Mars Helicopter</div>

In [128]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"Q&A with the Student Who Named Ingenuity, NASA's Mars Helicopter"

In [129]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'As a longtime fan of space exploration, Vaneeza Rupani appreciates the creativity and collaboration involved with trying to fly on another planet.'

### Featured Images


In [131]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)
browser

In [132]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [133]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')


In [134]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [135]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [136]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [137]:
df.to_html

<bound method DataFrame.to_html of                                     Mars            Earth
description                                              
Mars - Earth Comparison             Mars            Earth
Diameter:                       6,779 km        12,742 km
Mass:                    6.39 × 10^23 kg  5.97 × 10^24 kg
Moons:                                 2                1
Distance from Sun:        227,943,824 km   149,598,262 km
Length of Year:           687 Earth days      365.24 days
Temperature:                -87 to -5 °C      -88 to 58°C>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [138]:
# 1. Use browser to visit the URL 
url = 'https://data-class-mars-hemispheres.s3.amazonaws.com/Mars_Hemispheres/index.html'
browser.visit(url)

In [139]:
# # Parse the HTML
html = browser.html
html_soup = soup(html, 'html.parser')

# Find HTML tag that holds all links
tag_box =html_soup.find('div', class_='collapsible results')
tags = tag_box.find_all('div', class_='item')


In [140]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
# Iterate through tags 
for tag in tags:
#     tag.html
    # Create an empty dictionary 
#     hemispheres = {}
    # Clicking on Hemisphere link
    elem = tag.find('h3').text
    browser.click_link_by_partial_text(elem)
    # Parsing new generated browser
    html = browser.html
    img_soup = soup(html, 'html.parser')
    # Finding url
    url = img_soup.find('div', class_='downloads').find('a').get('href')
    # Adding base url
    img_url = f'https://data-class-mars-hemispheres.s3.amazonaws.com/Mars_Hemispheres/{url}'
    # Finding title
    title = img_soup.find('h2',class_='title').text
    # Creating Dictionary 
    hemispheres = {'img_url' : img_url,
                   'title': title}
    # Adding dictionary to list 
    hemisphere_image_urls.append(hemispheres)
    # Returning to previous page
    browser.back()


/Users/caseymarescot/opt/anaconda3/lib/python3.8/site-packages/splinter/driver/webdriver/__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


In [141]:
hemisphere_image_urls

[{'img_url': 'https://data-class-mars-hemispheres.s3.amazonaws.com/Mars_Hemispheres/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://data-class-mars-hemispheres.s3.amazonaws.com/Mars_Hemispheres/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://data-class-mars-hemispheres.s3.amazonaws.com/Mars_Hemispheres/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://data-class-mars-hemispheres.s3.amazonaws.com/Mars_Hemispheres/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]